# Diffusion
Based on the previous fourth tutorial for calculating bulk properties for unaries with the LAMMPS molecular dynamics simulation code, this tutorial covers the diffusion of hydrogen in bulk structures. Hydrogen diffusion is essential to understand the effects of hydrogen embrittlement highlighting the relevance of these simulation for engineering applications. 

In analogy to the previous exercies both codes are available as open-source software for the Linux operation system and can be installed from the `conda` package manager. See tutorial four for a detailed explanation of the installation. 

In addition, to the `Project` object from `pyiron_atomistics` also the numerical library `numpy` and the visualization library `matplotlib` are imported. The `Project` object behaves like a folder on the file system and adds the capability to create other `pyiron` objects like atomistic structures and simulation jobs. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from pyiron_atomistics import Project

A new `Project` object is created named `energy_volume_curve_md` to separate the calculation from the current tutorial and the previous. Any remaining calculation in the `energy_volume_curve_md` folder are removed using the `remove_jobs()` function, to provide a fresh start for every execution of the notebook.

In [ ]:
pr = Project("diffusion")
pr.remove_jobs(recursive=True, silently=True)

### Creating an Al supercell with interstitial H

#### Create Al sublattice

In [ ]:
Al_struct = pr.create_ase_bulk("Al", cubic=True)

#### Create H interstitial sublattice

Translate the H position by half the unit cell vectors.

In [ ]:
H_int = Al_struct.copy()
H_int[:] = "H"
H_int.positions += 0.5 * H_int.cell @ (1, 1, 1)

#### Create a 2x2x4 supercell for both lattices

In [ ]:
Al_struct.set_repeat([2, 2, 4])
H_int.set_repeat([2, 2, 4])

#### Occupy the H sublattice randomly with 10 H atoms 

In [ ]:
rand_int = np.arange(0, len(H_int))
np.random.shuffle(rand_int)
H_int = H_int[rand_int[:10]]

#### Combine Al and H sublattice in same supercell

In [ ]:
AlH_struct = Al_struct + H_int
AlH_struct.plot3d(particle_size=3)

## Perform a MD simulation of the AlH supercell using LAMMPS

### Setup the Lammps calculation

In [ ]:
job_lammps = pr.create_job(job_type=pr.job_type.Lammps, job_name= "AlH_lammps")
job_lammps.structure = AlH_struct
print(job_lammps.list_potentials())
job_lammps.potential = '1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1'
job_lammps.calc_md(temperature=600, n_ionic_steps=10000)

In [ ]:
job_lammps.run()

### Animate MD trajectories

In [ ]:
job_lammps.animate_structure(particle_size=1)

### Analyze trajectories in more detail

In [ ]:
H_indices = AlH_struct.select_index("H")
Al_indices = AlH_struct.select_index("Al")
H_indices

#### Plot Al trajectories in grey, H ones in red

In [ ]:
plt.plot(job_lammps.output.unwrapped_positions[:, :, 2], color="grey")
plt.plot(job_lammps.output.unwrapped_positions[:, H_indices, 2], color="red")
plt.xlabel("time [fs]")
plt.ylabel("positions [$\AA$]");

In [ ]:
md_struct = job_lammps.get_structure(iteration_step=-1)
md_struct.plot3d(particle_size=2)

## Analysis

### Plot Al-Al and H-H pair correlation function of the last MD structure

This is done using the `get_neighbors()` function

In [ ]:
neighbors_md = md_struct[Al_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_md.distances.flatten(), bins=50, alpha=0.3, density=True, label="Al-Al")
neighbors_md = md_struct[H_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_md.distances.flatten(), bins=50, alpha=0.3, density=True, label="H-H")
plt.xlabel('distance [A]')
plt.legend();

### Plot Al-Al and H-H pair correlation function after structure and cell relaxation 

In [ ]:
min_struct = job_lammps_minimize.get_structure(-1)
neighbors_min = min_struct[Al_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_min.distances.flatten(), bins=50, alpha=0.3, density=True, label="Al-Al")
neighbors_min = min_struct[H_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_min.distances.flatten(), bins=50, alpha=0.3, density=True, label="H-H")
plt.xlabel('distance [A]')
plt.legend();

### Compare Al-Al pair correlation before and after relaxation

In [ ]:
neighbors_min = min_struct[Al_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_min.distances.flatten(), bins=50, alpha=0.3, density=True, label="Al-Al min")
neighbors_md = md_struct[Al_indices].get_neighbors(num_neighbors=100)
plt.hist(neighbors_md.distances.flatten(), bins=50, alpha=0.3, density=True, label="Al-Al md")
plt.legend();